# Computing Eady Growth Rate
Following functions are modified after ESMValTool.
https://docs.esmvaltool.org/en/v2.5.0/recipes/recipe_eady_growth_rate.html

Analyses were done by using daily data to avoid the bias reported by Simmonds and Lim (2009)
https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2008GL036320

In [1]:
"""Diagnostic for PRIMAVERA Eady Growth Rate."""
import logging
import os
import sys
import command as cmd
import cartopy.crs as ccrs
import iris
import iris.analysis
import iris.cube
import iris.quickplot as qplt
import iris.util
import matplotlib.pyplot as plt
import numpy as np
from dask import array as da
from esmvalcore.preprocessor import (
    annual_statistics,
    extract_levels,
    regrid,
    seasonal_statistics,
)

from esmvaltool.diag_scripts.shared import (
    ProvenanceLogger,
    group_metadata,
    names,
    run_diagnostic,
)

logger = logging.getLogger().setLevel("Info".upper())

In [6]:
class EadyGrowthRate:
    """Class used to compute the Eady Growth Rate."""

    def __init__(self, config):
        """
        Set diagnostic parameters and constants.

        Parameters
        ----------
            config : dict
                Dictionary containing configuration settings.
        """
        self.cfg = config
        self.fill_value = 1e20
        """Fill Value."""
        self.ref_p = 1000.0
        """Reference Pressure [Pa]."""
        self.gravity = 9.80665
        """Gravity [m/s]."""
        self.con = 0.3098
        """Constant."""
        self.omega = 7.292e-5
        """Rotation of the Earth [rad/s]."""
        self.time_statistic = self.cfg['time_statistic']
        """Time statistic to perform."""
        self.input_dir = self.cfg['input_dir']

    def compute(self):
        """Compute Eady Growth Rate and either it's annual or seasonal mean."""
        temperature = iris.load_cube(f"{self.input_dir}/T.nc", 'ta')
        plev = temperature.coord('air_pressure')
    
        theta = self.potential_temperature(temperature, plev)
        del temperature
    
        geopotential = iris.load_cube(f"{self.input_dir}/z.nc", 'zg')
        brunt = self.brunt_vaisala_frq(theta, geopotential)
    
        lats = geopotential.coord('latitude')
        fcor = self.coriolis(lats, geopotential.shape)
    
        eastward_wind = iris.load_cube(f"{self.input_dir}/u.nc", 'ua')
        if eastward_wind.shape is not geopotential.shape:
            eastward_wind = regrid(eastward_wind, geopotential, scheme='linear')
    
        egr = self.eady_growth_rate(fcor, eastward_wind, geopotential, brunt)
    
        cube_egr = eastward_wind.copy(egr * 86400)
        cube_egr.standard_name = None
        cube_egr.long_name = 'eady_growth_rate'
        cube_egr.var_name = 'egr'
        cube_egr.units = 'day-1'
    
        # 時間統計の処理
        if self.time_statistic == 'annual_mean':
            cube_egr = annual_statistics(cube_egr)
            cube_egr = cube_egr.collapsed('time', iris.analysis.MEAN)
        elif self.time_statistic == 'seasonal_mean':
            cube_egr = seasonal_statistics(cube_egr)
            cube_egr = cube_egr.collapsed('time', iris.analysis.MEAN)
            self.seasonal_plots(cube_egr)  # alias引数を削除
        else:
            logger.info("Parameter time_statistic is not well set in the recipe."
                        "Must be 'annual_mean' or 'seasonal_mean'")
            sys.exit()
    
        self.save(cube_egr)  # dataとalias引数を削除


    def potential_temperature(self, temperature, plev):
        """Compute potential temperature.

        Parameters
        ----------
        temperature: iris.cube.Cube
            Cube of air temperature ta.
        plev: iris.coords.Coord
            Pressure level coordinates

        Returns
        -------
        theta: iris.cube.Cube
            Cube of potential temperature theta.
        """
        reference_pressure = iris.coords.AuxCoord(
            self.ref_p, long_name='reference_pressure', units='hPa')
        reference_pressure.convert_units(plev.units)
        pressure = (reference_pressure.points / plev.points)**(2 / 7)
        theta = temperature * iris.util.broadcast_to_shape(
            pressure, temperature.shape,
            temperature.coord_dims('air_pressure'))
        theta.long_name = 'potential_air_temperature'

        return theta

    @staticmethod
    def vertical_integration(var_x, var_y):
        """
        Vertical integration.

        Perform a non-cyclic centered finite-difference to integrate
        variable x with respect to variable y along pressure levels.

        Parameters
        ----------
        x: iris.cube.Cube
            Cube of variable x.
        y: iris.cube.Cube
            Cube of variable y.

        Returns
        -------
        dxdy: iris.cube.Cube
            Cube of variable integrated along pressure levels.
        """
        plevs = var_x.shape[1]

        dxdy_0 = (
            (var_x[:, 1, :, :].lazy_data() - var_x[:, 0, :, :].lazy_data()) /
            (var_y[:, 1, :, :].lazy_data() - var_y[:, 0, :, :].lazy_data()))

        dxdy_centre = ((var_x[:, 2:plevs, :, :].lazy_data() -
                        var_x[:, 0:plevs - 2, :, :].lazy_data()) /
                       (var_y[:, 2:plevs, :, :].lazy_data() -
                        var_y[:, 0:plevs - 2, :, :].lazy_data()))

        dxdy_end = ((var_x[:, plevs - 1, :, :].lazy_data() -
                     var_x[:, plevs - 2, :, :].lazy_data()) /
                    (var_y[:, plevs - 1, :, :].lazy_data() -
                     var_y[:, plevs - 2, :, :].lazy_data()))

        bounds = [dxdy_end, dxdy_0]
        stacked_bounds = da.stack(bounds, axis=1)
        total = [dxdy_centre, stacked_bounds]

        # Concatenate arrays where the last slice is dxdy_0
        dxdy = da.concatenate(total, axis=1)

        # Move dxdy_0 to the beggining of the array
        dxdy = da.roll(dxdy, 1, axis=1)

        return dxdy

    def brunt_vaisala_frq(self, theta, geopotential):
        """Compute Brunt-Väisälä frequency.

        Parameters
        ----------
        theta: iris.cube.Cube
            Cube of potential temperature.
        geopotential: iris.cube.Cube
            Cube of variable zg.

        Returns
        -------
        brunt: da.array
            Array containing Brunt-Väisälä frequency.
        """
        dthdz = self.vertical_integration(theta, geopotential)
        dthdz = da.where(dthdz > 0, dthdz, 0)
        buoy = (self.gravity / theta.lazy_data()) * dthdz
        brunt = da.sqrt(buoy)
        brunt = da.where(brunt != 0, brunt, self.fill_value)

        return brunt

    def coriolis(self, lats, ndim):
        """Compute Coriolis force.

        Parameters
        ----------
        lats: iris.coord.Coord
            Latitude coordinate.
        ndim: int
            Number of dimension.

        Returns
        -------
        fcor: da.array
            Array containing Coriolis force.
        """
        fcor = 2.0 * self.omega * np.sin(np.radians(lats.points))
        fcor = fcor[np.newaxis, np.newaxis, :, np.newaxis]
        fcor = da.broadcast_to(fcor, ndim)

        return fcor

    def eady_growth_rate(self, fcor, eastward_wind, geopotential, brunt):
        """Compute Eady Growth Rate.

        Parameters
        ----------
        fcor: da.array
            Array containing Coriolis force.
        eastward_wind: iris.cube.Cube
            Cube containing variable ua.
        geopotential: iris.cube.Cube
            Cube containing variable zg.
        brunt: da.array
            Array containing Brunt-Väisäla frequency

        Returns
        -------
        egr: da.array
            Array containing Eady Growth Rate.
        """
        dudz = self.vertical_integration(eastward_wind, geopotential)
        egr = self.con * abs(fcor) * abs(dudz) / brunt

        return egr

    def seasonal_plots(self, egr):
        """Plot seasonal Eady Growth rate values."""
        try:
            levels = self.cfg['plot_levels']
        except KeyError:
            logger.info("Parameter plot_levels is not set in the recipe."
                        "Plotting all pressure levels instead.")
            levels = egr.coord('air_pressure').points
        for level in levels:
            cube = extract_levels(egr, level, scheme='linear')
            crs_latlon = ccrs.PlateCarree()
            axes = plt.axes(projection=ccrs.PlateCarree())
            axes.coastlines(linewidth=1, color='black')
            # North Atlantic
            axes.set_extent((-90.0, 30.0, 20.0, 80.0), crs=crs_latlon)
            axes.set_yticks(np.linspace(25, 75, 6))
            qplt.contourf(cube, levels=np.arange(0, 1.1, 0.05))
            extension = self.cfg['output_file_type']
            diagnostic = self.cfg['script']
            plotname = f'{diagnostic}_{str(int(level))}.{extension}'
            plt.savefig(os.path.join(self.cfg[names.PLOT_DIR], plotname))
            plt.close()

    def save(self, egr):
            """Save results."""
            output_name = 'eady_growth_rate.nc'
            output_file = os.path.join(self.cfg[names.WORK_DIR], output_name)
            iris.save(egr, output_file)

def main(cfg):
    """Run Eady Growth Rate diagnostic."""
    EadyGrowthRate(cfg).compute()

In [7]:
import subprocess
def command(cmd):
    # Pythonでシェルコマンドを動かし、結果を取得する関数
    # 参考 : https://qiita.com/inatatsu_csg/items/40b11701d256a84a0510 
    process = (subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True).communicate()[0]).decode('utf-8')[:-1]#.split("\n")
    return process

In [8]:
exp_list = ["LGM.miroc_glomapice_anomtopo_t42.20230831", "LGM.miroc_anomtopo_t42.20230831","LGM.glomap_anomtopo_t42.20230831"]


In [9]:
#for expname in exp_list:
for expname in ["LGM.glomap_anomtopo_t42.20230831"]:
    dir = f'/data44/kanon/kino2023grl/{expname}'
    for var in ["T","z","u"]:
        if not os.path.exists(f'{dir}/cat/{var}.nc'):
            print(f'ngtcf {dir}/cat/{var} {dir}/cat/{var}.nc')
        
    cfg = {
        'input_dir': f'{dir}/cat',
        'time_statistic': 'annual_mean',  # 'annual_mean' か 'seasonal_mean'
        'plot_levels': [85000, 50000],  # 描画する気圧レベルのリスト
        'output_file_type': 'png',  # 出力ファイルの種類（例：'png', 'pdf'）
        'script': 'eady_growth_rate',  # スクリプト名
        'plot_dir': '/data44/kanon/kino2023grl/pic',  # プロットを保存するディレクトリ
        'work_dir': f'{dir}/clm/ann'  # 出力データを保存するディレクトリ
    }

    main(cfg)

/home/kanon/.pyenv/versions/mambaforge-22.9.0-0/envs/py2023/lib/python3.11/site-packages/iris/coords.py:2237: UserWarning: Cannot check if coordinate is contiguous: Invalid operation for 'year', with 0 bound(s). Contiguous bounds are only defined for 1D coordinates with 2 bounds. Metadata may not be fully descriptive for 'year'. Ignoring bounds.
  warnings.warn(msg.format(str(exc), self.name()))


In [15]:
dir = f'/data44/kanon/kino2023grl'
for expname in exp_list:
    file = f'{dir}/{expname}/clm/ann/eady_growth_rate'
    print(f'cdo sellevel,850 {dir}/{expname}/clm/ann/eady_growth_rate.nc {dir}/{expname}/clm/ann/egr850.nc')

cdo sellevel,850 /data44/kanon/kino2023grl/LGM.miroc_glomapice_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.miroc_glomapice_anomtopo_t42.20230831/clm/ann/egr850.nc
cdo sellevel,850 /data44/kanon/kino2023grl/LGM.miroc_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.miroc_anomtopo_t42.20230831/clm/ann/egr850.nc
cdo sellevel,850 /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/clm/ann/egr850.nc


In [ ]:
ngted -e aitm3:c"STDPL18" egr

In [82]:
!/home/acauquoin/local/bin/ncgt -v egr /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_glomapice_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_glomapice_anomtopo_t42.20230831/clm/ann/egr
!/home/acauquoin/local/bin/ncgt -v egr /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_anomtopo_t42.20230831/clm/ann/egr
!/home/acauquoin/local/bin/ncgt -v egr /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.glomap_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.glomap_anomtopo_t42.20230831/clm/ann/egr

ncgt: /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_glomapice_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc: No such file or directory
ncgt: /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.miroc_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc: No such file or directory
ncgt: /data44/kanon/kino2023grl/LGM.glomap_anomtopo_t42.20230831/LGM.glomap_anomtopo_t42.20230831/clm/ann/eady_growth_rate.nc: No such file or directory
